# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

In [1]:
!pip install numpy==1.26.4 nltk==3.9.1 scikit-learn==1.6.1 gensim==4.3.3 setuptools

     ---------------------------------------- 1.5/1.5 MB 8.7 MB/s eta 0:00:00
     --------------------------------------- 11.1/11.1 MB 21.8 MB/s eta 0:00:00
     --------------------------------------- 24.0/24.0 MB 21.8 MB/s eta 0:00:00
     ------------------------------------- 301.8/301.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
  Attempting uninstall: gensim
    F

In [1]:
# Environment Setup and Validation
import sys
print(f"Python version: {sys.version}")

# Required package versions
required_packages = {
    'numpy': '1.26.4',
    'nltk': '3.9.1',
    'scikit-learn': '1.6.1',
    'gensim': '4.3.3'
}

def validate_environment():
    import pkg_resources
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for package, min_version in required_packages.items():
        if package not in installed:
            print(f"❌ {package} not found!")
            return False
        if pkg_resources.parse_version(installed[package]) < pkg_resources.parse_version(min_version):
            print(f"❌ {package} version {installed[package]} is below minimum {min_version}")
            return False
        print(f"✓ {package} version {installed[package]}")
    return True

assert validate_environment(), "Please install required packages with correct versions"

Python version: 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]
✓ numpy version 1.26.4
✓ nltk version 3.9.1
✓ scikit-learn version 1.6.1
✓ gensim version 4.3.3


# 1 $n$-gram Language Model

In [ ]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/lm/test.txt

## 1.1 Building vocabulary

### Instructions:
1. Implement a Vocabulary class that:
   - Handles OOV words (tokens occurring < 3 times)
   - Includes special tokens \<UNK>, \<START>, \<END>
   - Provides word2idx and idx2word mappings
2. Expected vocabulary size: 24,067 words

### Code

In [2]:
# 1.1 Building vocabulary
import nltk
from collections import Counter
from typing import List, Dict

nltk.download('punkt_tab')

class Vocabulary:
    def __init__(self):
        self.word2idx = {'<UNK>': 0, '<START>': 1, '<END>': 2}
        self.idx2word = ['<UNK>', '<START>', '<END>']
        self.word_counts = {}
        
    def build_vocab(self, filepath: str, min_freq: int = 3) -> None:
        """Build vocabulary from training file
        
        Args:
            filepath: Path to training file
            min_freq: Minimum frequency threshold for words
        """
        

        words = nltk.tokenize.word_tokenize(open(filepath).read().lower())
        self.word_counts = Counter(words)
        
        for word, count in self.word_counts.items():
            if count >= min_freq:
                self.word2idx[word] = len(self.word2idx)
                self.idx2word.append(word)
        
    
    def __len__(self) -> int:
        return len(self.word2idx)
    
    def get_id(self, word: str) -> int:
        """Get ID for word (returns UNK if not in vocab)"""

        if word in self.word2idx:
            return self.word2idx[word]
        else:
            return self.word2idx['<UNK>']
    
    def get_word(self, idx: int) -> str:
        """Get word for ID"""

        return self.idx2word[idx]

# Test vocabulary
vocab = Vocabulary()
vocab.build_vocab('data/lm/train.txt')
print(f"Vocabulary size: {len(vocab)}")
assert len(vocab) == 24067, "Vocabulary size should be 24,067"

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Vocabulary size: 24067


### Discussion

Discuss the number of parameters in n-gram models here

## 1.2 $n$-gram Language Modeling

### Instructions:
1. Implement an n-gram language model that:
   - Counts n-grams from training data
   - Computes probabilities
   - Calculates perplexity
2. Test with both unigram and bigram models
3. Report perplexity on train and dev sets

### Code

In [6]:
import numpy as np
from collections import defaultdict
from typing import List, Tuple

class LanguageModel:
    def __init__(self, vocab: Vocabulary, n: int):
        """Initialize n-gram language model
        
        Args:
            vocab: Vocabulary object
            n: Order of n-gram model
        """

        self.vocab = vocab
        self.n = n
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)
        
    def get_ngrams(self, tokens: List[str], n: int) -> List[Tuple[str]]:
        """Extract n-grams from token sequence"""

        n_grams = []
        for i in range(len(tokens) - n + 1):
            n_grams.append(tuple(tokens[i:i+n]))
        return n_grams
        
    def train(self, filepath: str) -> None:
        """Train n-gram language model"""

        tokens = []
        with open(filepath) as f:
            for line in f:
                tokens_in_line = nltk.tokenize.word_tokenize(line.lower())
                tokens_in_line = [token if token in self.vocab.idx2word else '<UNK>' for token in tokens_in_line]
                tokens.extend((self.n - 1) * ['<START>'] + tokens_in_line + (self.n - 1) * ['<END>'])
        n_grams = self.get_ngrams(tokens, self.n)
        self.ngram_counts = Counter(n_grams)
        context = self.get_ngrams(tokens, self.n - 1)
        self.context_counts = Counter(context)


                
    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get probability of word given context"""

        return self.ngram_counts[context + (word,)]  / self.context_counts[context]  
    
    def perplexity(self, filepath: str) -> float:
        """Calculate perplexity on given text"""

        L = 0
        line_counter = 0
        with open(filepath) as f:
            for line in f:
                line_counter += 1
                tokens = []
                tokens_in_line = nltk.tokenize.word_tokenize(line.lower())
                tokens_in_line = [token if token in self.vocab.idx2word else '<UNK>' for token in tokens_in_line]
                tokens.extend((self.n - 1) * ['<START>'] + tokens_in_line + (self.n - 1) * ['<END>'])
                for i in range(len(tokens) - self.n + 1):
                    context = tuple(tokens[i:i+self.n-1])
                    word = tokens[i+self.n-1]
                    L += np.log(self.get_prob(word, context))
        L /= line_counter
        return 2 ** (-L)
                

# Test language model
print("Testing Unigram model")
lm = LanguageModel(vocab, n=1)
lm.train('data/lm/train.txt')
train_perplexity = lm.perplexity('data/lm/train.txt')
print(f"Perplexity on train: {train_perplexity:.2f}")
assert train_perplexity > 0, "Perplexity should be positive"
dev_perplexity = lm.perplexity('data/lm/dev.txt')
print(f"Perplexity on dev: {dev_perplexity:.2f}")
assert dev_perplexity > 0, "Perplexity should be positive"
print('\n')

print("Testing Bigram model")
lm = LanguageModel(vocab, n=2)
lm.train('data/lm/train.txt')
train_perplexity = lm.perplexity('data/lm/train.txt')
print(f"Perplexity on train: {train_perplexity:.2f}")
assert train_perplexity > 0, "Perplexity should be positive"
dev_perplexity = lm.perplexity('data/lm/dev.txt')
print(f"Perplexity on dev: {dev_perplexity:.2f}")
assert dev_perplexity > 0, "Perplexity should be positive"

Testing Unigram model
Perplexity on train: 36743349108360492568306221912112310642028820658388992.00
Perplexity on dev: 13410776981611227004933209290437282562544059864317952.00


Testing Bigram model
Perplexity on train: 91658798081712150559623617708032000.00


C:\Users\user\AppData\Local\Temp\ipykernel_17668\182068893.py:63: RuntimeWarning: divide by zero encountered in log
  L += np.log(self.get_prob(word, context))


Perplexity on dev: inf


### Discussion

Compare unigram and bigram model performance here

Answer:

The unigram model assigns probabilities purely based on word frequencies. If the training data has a large vocabulary with many rare words, probabilities for individual words become extremely small.
Numerical Instability: Multiplying many small probabilities (or summing log-probabilities) leads to underflow/overflow.


As we have not introduced smoothing, some context words in dev set do not exist in the model's context_counts dictionary, leading to "divide by zero" warning.
For bigrams in the dev set that never appeared in training,  𝑝(𝑤∣𝑐𝑜𝑛𝑡𝑒𝑥𝑡) = 0/0 -> log(0) = −inf -> 2^(-(-inf)) = inf
So perplexity on dev test becomes inf

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing

### Code

In [7]:
class AddOneLanguageModel(LanguageModel):
    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get smoothed probability using add-one smoothing"""
        return (self.ngram_counts[context + (word,)] + 1) / (self.context_counts[context] + len(self.vocab)) 

# Test add-one smoothing
add_one_lm = AddOneLanguageModel(vocab, n=2)
add_one_lm.train('data/lm/train.txt')
print(f"Add-one perplexity on dev: {add_one_lm.perplexity('data/lm/dev.txt'):.2f}")

Add-one perplexity on dev: 20445536530650362214269891051222707355811898949829518163968.00


#### Discussion

Analyze how add-one smoothing affects the model performance

#### 1.3.2 Add-k smoothing

### Instructions:
1. Implement add-k smoothing with configurable k
2. Try k values: 0.1, 0.5, 1.0

##### Code

In [8]:
class AddKLanguageModel(LanguageModel):
    def __init__(self, vocab: Vocabulary, n: int, k: float):
        super().__init__(vocab, n)
        self.k = k
        
    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get smoothed probability using add-k smoothing"""
        return (self.ngram_counts[context + (word,)] + self.k) / (self.context_counts[context] + self.k * len(self.vocab))

# Test add-k smoothing with different k values
k_values = [0.1, 0.5, 1.0]
for k in k_values:
    add_k_lm = AddKLanguageModel(vocab, n=2, k=k)
    add_k_lm.train('data/lm/train.txt')
    print(f"Add-{k} perplexity on dev: {add_k_lm.perplexity('data/lm/dev.txt'):.2f}")

Add-0.1 perplexity on dev: 3715008495159369346337322801386625080596916040892416.00
Add-0.5 perplexity on dev: 98912615932611895189546270179767627933763998264149934080.00
Add-1.0 perplexity on dev: 20445536530650362214269891051222707355811898949829518163968.00


##### Discussion

Compare performance with different k values

### 1.3.3 Linear Interpolation

### Instructions:
1. Implement linear interpolation of unigram, bigram, and trigram models
2. Initial lambdas: [0.1, 0.3, 0.6]

#### Code

In [12]:
class InterpolatedLanguageModel:
    def __init__(self, vocab: Vocabulary, lambdas: List[float]):
        """
        Args:
            vocab: Vocabulary object
            lambdas: List of interpolation weights (should sum to 1)
        """
        assert abs(sum(lambdas) - 1.0) < 1e-6, "Lambdas must sum to 1"
        self.vocab = vocab
        self.lambdas = lambdas
        self.models = [AddOneLanguageModel(vocab, n=i+1) for i in range(len(lambdas))]
        
    def train(self, filepath: str) -> None:
        """Train all n-gram models"""
        for model in self.models:
            model.train(filepath)
            
    def get_prob(self, word: str, context: Tuple[str]) -> float:
        """Get interpolated probability"""
        return sum(lambda_ * model.get_prob(word, context) for lambda_, model in zip(self.lambdas, self.models))
    
    def perplexity(self, filepath: str) -> float:
        """Calculate perplexity using interpolated probabilities"""
        
        L = 0
        line_counter = 0
        with open(filepath) as f:
            for line in f:
                line_counter += 1
                tokens = []
                tokens_in_line = nltk.tokenize.word_tokenize(line.lower())
                tokens_in_line = [token if token in self.vocab.idx2word else '<UNK>' for token in tokens_in_line]
                tokens.extend((len(self.lambdas) - 1) * ['<START>'] + tokens_in_line + (len(self.lambdas) - 1) * ['<END>'])
                for i in range(len(tokens) - len(self.lambdas) + 1):
                    context = tuple(tokens[i:i+len(self.lambdas)-1])
                    word = tokens[i+len(self.lambdas)-1]
                    L += np.log(self.get_prob(word, context))
        L /= line_counter
        return 2 ** (-L)
    
    
# Test interpolated model
lambdas = [0.1, 0.3, 0.6]  # Example weights for unigram, bigram, trigram
interpolated_lm = InterpolatedLanguageModel(vocab, lambdas)
interpolated_lm.train('data/lm/train.txt')
print(f"Interpolated perplexity on dev: {interpolated_lm.perplexity('data/lm/dev.txt'):.2f}")

Interpolated perplexity on dev: 85412133823217001061792131807045925146780616670404438911629635304275574784.00


#### Discussion

Analyze the effectiveness of interpolation

##### 1.3.4 Optimization

#### Discussion

# 2 Word Vectors

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

## 2.1 Find most similar word
### Instructions:
1. Implement function to find similar words using cosine similarity
2. Expected similarity scores should be between 0 and 1
3. Return top 3 similar words for each query

In [ ]:
def find_most_similar(word: str, wv_from_bin) -> List[Tuple[str, float]]:
    """Find most similar words using cosine similarity
    
    Args:
        word: Query word
        wv_from_bin: Loaded word vectors
        
    Returns:
        List of (word, similarity) tuples
    """
    # TODO: Implement similarity search
    pass

test_words = [
    'ubiquitous',
    'serendipity',
    'melancholy',
    'paradox',
    'ethereal',
    'cacophony'
]
for word in test_words:
    similar_words = find_most_similar(word, wv_from_bin)
    print(f"\nMost similar to '{word}':")
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity:.3f}")
        assert 0 <= similarity <= 1, f"Invalid similarity score: {similarity}"

## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- France : French :: England : ?:
- France : wine :: England : ?

In [ ]:
def find_analogy(a: str, b: str, c: str, wv_from_bin) -> List[Tuple[str, float]]:
    """Find word analogies using vector arithmetic
    
    Args:
        a, b, c: Words for analogy a:b :: c:?
        wv_from_bin: Loaded word vectors
        
    Returns:
        List of (word, similarity) tuples
    """
    # TODO: Implement analogy finding
    pass

# Test analogies
analogies = [
    ('king', 'queen', 'man'), 
    ('paris', 'france', 'rome'),
    ('france', 'french', 'germany'),
    ('london', 'england', 'paris'),
    ('cat', 'kitten', 'dog'),
    ('rich', 'poor', 'strong')
]
for a, b, c in analogies:
    print(f"\nAnalogy: {a}:{b} :: {c}:?")
    results = find_analogy(a, b, c, wv_from_bin)
    for word, similarity in results:
        print(f"{word}: {similarity:.3f}")
        assert 0 <= similarity <= 1, f"Invalid similarity score: {similarity}"

# 3 Sentiment analysis

In [ ]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2025/main/assignments/A1/data/classification/test-blind.txt

## 3.1 Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

def load_data(filepath: str) -> Tuple[List[str], List[int]]:
    """Load text and labels from file"""
    texts, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            label, text = line.strip().split('\t')
            texts.append(text)
            labels.append(int(label))
    return texts, labels

class SentimentClassifier:
    def __init__(self, feature_type: str = 'unigram'):
        """
        Args:
            feature_type: One of 'unigram', 'bigram', or 'glove'
        """
        self.feature_type = feature_type
        self.vectorizer = None
        self.classifier = LogisticRegression(random_state=42)
        
    def extract_features(self, texts: List[str]) -> np.ndarray:
        """Extract features based on feature_type"""
        if self.feature_type == 'unigram':
            # TODO: Implement unigram feature extraction
            pass
        elif self.feature_type == 'bigram':
            # TODO: Implement bigram feature extraction 
            pass
        elif self.feature_type == 'glove':
            # TODO: Implement glove feature extraction
            # Average word vectors for each text
            pass
        else:
            raise ValueError(f"Invalid feature type: {self.feature_type}")
            
    def train(self, train_texts: List[str], train_labels: List[int]) -> None:
        """Train sentiment classifier"""
        # TODO: Implement training
        pass
        
    def predict(self, texts: List[str]) -> np.ndarray:
        """Predict sentiment labels"""
        # TODO: Implement prediction
        pass
    
    def evaluate(self, texts: List[str], labels: List[int]) -> Dict:
        """Evaluate classifier performance"""
        predictions = self.predict(texts)
        return classification_report(labels, predictions, output_dict=True)

# Train and evaluate models with different features
train_texts, train_labels = load_data('data/classification/train.txt')
dev_texts, dev_labels = load_data('data/classification/dev.txt')

results = {}
for feature_type in ['unigram', 'bigram', 'glove']:
    classifier = SentimentClassifier(feature_type)
    classifier.train(train_texts, train_labels)
    results[feature_type] = classifier.evaluate(dev_texts, dev_labels)
    
# Print results
for feature_type, metrics in results.items():
    print(f"\n{feature_type.upper()} Results:")
    print(f"Precision: {metrics['weighted avg']['precision']:.3f}")
    print(f"Recall: {metrics['weighted avg']['recall']:.3f}")
    print(f"F1-score: {metrics['weighted avg']['f1-score']:.3f}")

Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     |           |        |          |
| bigram      |           |        |          |
| GloVe       |           |        |          |

## 3.2 Better Feature